In [2]:
import torch
from torch import nn
import sys
sys.path.insert(0, "..")
from helper import train, test
from data import get_dataloader
import torch.nn.functional as F
from time import time

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps"

# AlexNet
![AlexNet](https://miro.medium.com/max/1200/1*3B8iO-se13vA2QfZ4OBRSw.png)

*The dimensions of the final Fully connected layers have been modified since AlexNet was designed for ImageNet (227x227x3) images but CIFAR10 has 32x32x3 images.

In [4]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            # Input Dimension 32x32x3
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=1),
            nn.ReLU(),
            # 22x22x96
            nn.MaxPool2d(kernel_size=3, stride=2),
            # 10x10x96

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2),
            nn.ReLU(),
            # 10x10x256
            nn.MaxPool2d(kernel_size=3, stride=2),
            # 4x4x256

            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(),
            # 4x4x384
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(),
            # 4x4x384
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),

            # 4x4x256
            nn.MaxPool2d(kernel_size=3, stride=2),
            # 1x1x256
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )
        

    def forward(self, x):
        
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits
        


In [5]:
# get the data
train_dl, test_dl = get_dataloader("cifar", batch_size=128)
# Training the model
model = AlexNet().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

epochs = 10
start = time()
for t in range(epochs):
    print(f"Epoch {t+1}\n---")
    train(train_dl, model, loss_fn, optimizer, device)
    test(test_dl, model, loss_fn, device)
print(f"Total time taken: {(time()-start):>0.1f} seconds")
    


Files already downloaded and verified
Files already downloaded and verified
Epoch 1
---
loss: 2.307439  [    0/50000]
loss: 1.791200  [12800/50000]
loss: 1.824623  [25600/50000]
loss: 1.586353  [38400/50000]
Train Error: 
 Accuracy: 32.6%
Test Error: 
 Accuracy: 43.0%, Avg loss: 1.541428 

Epoch 2
---
loss: 1.551978  [    0/50000]
loss: 1.358973  [12800/50000]
loss: 1.422033  [25600/50000]
loss: 1.373563  [38400/50000]
Train Error: 
 Accuracy: 49.0%
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.264790 

Epoch 3
---
loss: 1.341837  [    0/50000]
loss: 1.211029  [12800/50000]
loss: 1.244110  [25600/50000]
loss: 1.257892  [38400/50000]
Train Error: 
 Accuracy: 57.1%
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.125776 

Epoch 4
---
loss: 1.146798  [    0/50000]
loss: 1.063335  [12800/50000]
loss: 1.080281  [25600/50000]
loss: 1.140488  [38400/50000]
Train Error: 
 Accuracy: 62.4%
Test Error: 
 Accuracy: 62.9%, Avg loss: 1.050735 

Epoch 5
---
loss: 1.015060  [    0/50000]
loss: 0.963046  [12800